In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer

In [2]:
# Read the CSV files into DataFrames
mar22_df = pd.read_csv('mar22_unicorn.csv')
nov22_df = pd.read_csv('nov22_unicorn.csv')

# Merge the DataFrames
merged_df = pd.merge(mar22_df, nov22_df, on='Company', how='outer')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_unicorn.csv', index=False)


In [3]:
merged_df.head()

,Company,Valuation,Date Joined_x,Industry_x,City_x,Country/Region,Continent,Year Founded,Funding,Select Investors_x,Unnamed: 0,Valuation ($B),Date Joined_y,Country,City_y,Industry_y,Select Investors_y
0,Bytedance,180.0,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012.0,$8B,"Sequoia Capital China, SIG Asia Investments, S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SpaceX,100.0,2012-12-01,Other,Hawthorne,United States,North America,2002.0,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",1.0,$127,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100.0,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008.0,$2B,"Tiger Global Management, Sequoia Capital China...",2.0,$100,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95.0,2014-01-23,FinTech,San Francisco,United States,North America,2010.0,$2B,"Khosla Ventures, LowercaseCapital, capitalG",3.0,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46.0,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005.0,$4B,"Institutional Venture Partners, Sequoia Capita...",98.0,$6.7,12/12/2011,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita..."


In [4]:
merged_df["Date Joined"] = merged_df["Date Joined_x"].combine_first(merged_df["Date Joined_y"])
merged_df.drop(["Date Joined_x", "Date Joined_y"], axis=1, inplace=True)

merged_df["Industry"] = merged_df["Industry_x"].combine_first(merged_df["Industry_y"])
merged_df.drop(["Industry_x", "Industry_y"], axis=1, inplace=True)

merged_df["City"] = merged_df["City_x"].combine_first(merged_df["City_y"])
merged_df.drop(["City_x", "City_y"], axis=1, inplace=True)

merged_df["Select Investors"] = merged_df["Select Investors_x"].combine_first(merged_df["Select Investors_y"])
merged_df.drop(["Select Investors_x", "Select Investors_y"], axis=1, inplace=True)

merged_df["Region"] = merged_df["Country/Region"].combine_first(merged_df["Country"])
merged_df.drop(["Country/Region", "Country"], axis=1, inplace=True)

merged_df.rename(columns={'Funding': 'Funding ($B)'}, inplace=True)

columns_to_drop = ['Unnamed: 0']
merged_df = merged_df.drop(columns=columns_to_drop)
merged_df = merged_df[~merged_df.index.duplicated(keep="Company")]
merged_df.head()

,Company,Valuation,Continent,Year Founded,Funding ($B),Valuation ($B),Date Joined,Industry,City,Select Investors,Region
0,Bytedance,180.0,Asia,2012.0,$8B,NaN,2017-04-07,Artificial intelligence,Beijing,"Sequoia Capital China, SIG Asia Investments, S...",China
1,SpaceX,100.0,North America,2002.0,$7B,$127,2012-12-01,Other,Hawthorne,"Founders Fund, Draper Fisher Jurvetson, Rothen...",United States
2,SHEIN,100.0,Asia,2008.0,$2B,$100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,"Tiger Global Management, Sequoia Capital China...",China
3,Stripe,95.0,North America,2010.0,$2B,$95,2014-01-23,FinTech,San Francisco,"Khosla Ventures, LowercaseCapital, capitalG",United States
4,Klarna,46.0,Europe,2005.0,$4B,$6.7,2011-12-12,Fintech,Stockholm,"Institutional Venture Partners, Sequoia Capita...",Sweden


In [5]:
merged_df.set_index('Company', inplace=True)

In [6]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1241 entries, Bytedance to Unstoppable Domains
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Valuation         1077 non-null   float64
 1   Continent         1077 non-null   object 
 2   Year Founded      1077 non-null   float64
 3   Funding ($B)      1077 non-null   object 
 4   Valuation ($B)    1203 non-null   object 
 5   Date Joined       1241 non-null   object 
 6   Industry          1241 non-null   object 
 7   City              1223 non-null   object 
 8   Select Investors  1241 non-null   object 
 9   Region            1241 non-null   object 
dtypes: float64(2), object(8)
memory usage: 106.6+ KB


In [7]:
desired_order = [
    "Date Joined", "Year Founded", 'City', 'Region', 'Continent',
    "Select Investors", "Valuation ($B)", "Funding ($B)", 'Industry'
]

merged_df = merged_df[desired_order]
merged_df.head()


,Date Joined,Year Founded,City,Region,Continent,Select Investors,Valuation ($B),Funding ($B),Industry
Company,,,,,,,,,
Bytedance,2017-04-07,2012.0,Beijing,China,Asia,"Sequoia Capital China, SIG Asia Investments, S...",NaN,$8B,Artificial intelligence
SpaceX,2012-12-01,2002.0,Hawthorne,United States,North America,"Founders Fund, Draper Fisher Jurvetson, Rothen...",$127,$7B,Other
SHEIN,2018-07-03,2008.0,Shenzhen,China,Asia,"Tiger Global Management, Sequoia Capital China...",$100,$2B,E-commerce & direct-to-consumer
Stripe,2014-01-23,2010.0,San Francisco,United States,North America,"Khosla Ventures, LowercaseCapital, capitalG",$95,$2B,FinTech
Klarna,2011-12-12,2005.0,Stockholm,Sweden,Europe,"Institutional Venture Partners, Sequoia Capita...",$6.7,$4B,Fintech


In [8]:
merged_df['Valuation ($B)'] = pd.to_numeric(merged_df['Valuation ($B)'].str.replace('$', ''), errors='coerce')

merged_df.head()

C:\Users\adiat\AppData\Local\Temp\ipykernel_21380\1219785812.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged_df['Valuation ($B)'] = pd.to_numeric(merged_df['Valuation ($B)'].str.replace('$', ''), errors='coerce')


,Date Joined,Year Founded,City,Region,Continent,Select Investors,Valuation ($B),Funding ($B),Industry
Company,,,,,,,,,
Bytedance,2017-04-07,2012.0,Beijing,China,Asia,"Sequoia Capital China, SIG Asia Investments, S...",NaN,$8B,Artificial intelligence
SpaceX,2012-12-01,2002.0,Hawthorne,United States,North America,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.0,$7B,Other
SHEIN,2018-07-03,2008.0,Shenzhen,China,Asia,"Tiger Global Management, Sequoia Capital China...",100.0,$2B,E-commerce & direct-to-consumer
Stripe,2014-01-23,2010.0,San Francisco,United States,North America,"Khosla Ventures, LowercaseCapital, capitalG",95.0,$2B,FinTech
Klarna,2011-12-12,2005.0,Stockholm,Sweden,Europe,"Institutional Venture Partners, Sequoia Capita...",6.7,$4B,Fintech


In [9]:
column_mean = merged_df["Valuation ($B)"].mean()
print(column_mean)

3.220149625935162


In [10]:
mean_column2 = merged_df["Valuation ($B)"].mean()
merged_df["Valuation ($B)"].fillna(mean_column2, inplace=True)

mean_column2 = merged_df["Year Founded"].mean()
merged_df["Year Founded"].fillna(mean_column2, inplace=True)

merged_df.head()

,Date Joined,Year Founded,City,Region,Continent,Select Investors,Valuation ($B),Funding ($B),Industry
Company,,,,,,,,,
Bytedance,2017-04-07,2012.0,Beijing,China,Asia,"Sequoia Capital China, SIG Asia Investments, S...",3.22015,$8B,Artificial intelligence
SpaceX,2012-12-01,2002.0,Hawthorne,United States,North America,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.00000,$7B,Other
SHEIN,2018-07-03,2008.0,Shenzhen,China,Asia,"Tiger Global Management, Sequoia Capital China...",100.00000,$2B,E-commerce & direct-to-consumer
Stripe,2014-01-23,2010.0,San Francisco,United States,North America,"Khosla Ventures, LowercaseCapital, capitalG",95.00000,$2B,FinTech
Klarna,2011-12-12,2005.0,Stockholm,Sweden,Europe,"Institutional Venture Partners, Sequoia Capita...",6.70000,$4B,Fintech


In [11]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1241 entries, Bytedance to Unstoppable Domains
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date Joined       1241 non-null   object 
 1   Year Founded      1241 non-null   float64
 2   City              1223 non-null   object 
 3   Region            1241 non-null   object 
 4   Continent         1077 non-null   object 
 5   Select Investors  1241 non-null   object 
 6   Valuation ($B)    1241 non-null   float64
 7   Funding ($B)      1077 non-null   object 
 8   Industry          1241 non-null   object 
dtypes: float64(2), object(7)
memory usage: 97.0+ KB


In [12]:
def extract_numeric_value(value):
    match = re.search(r'\$(\d+(\.\d+)?)([BM]?)', str(value))
    if match:
        number = float(match.group(1))
        suffix = match.group(3)
        if suffix == 'B':
            number *= 1e9  # Convert to billions
        elif suffix == 'M':
            number *= 1e6  # Convert to millions
        return number
    else:
        return None

merged_df['Funding_Num'] = merged_df["Funding ($B)"].apply(extract_numeric_value)

mean_value = merged_df['Funding_Num'].mean()

imputer = SimpleImputer(strategy='constant', fill_value=mean_value)
merged_df['Funding_Num'] = imputer.fit_transform(merged_df[['Funding_Num']])

merged_df["Funding_Num"].head()
merged_df.head()

,Date Joined,Year Founded,City,Region,Continent,Select Investors,Valuation ($B),Funding ($B),Industry,Funding_Num
Company,,,,,,,,,,
Bytedance,2017-04-07,2012.0,Beijing,China,Asia,"Sequoia Capital China, SIG Asia Investments, S...",3.22015,$8B,Artificial intelligence,8.000000e+09
SpaceX,2012-12-01,2002.0,Hawthorne,United States,North America,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.00000,$7B,Other,7.000000e+09
SHEIN,2018-07-03,2008.0,Shenzhen,China,Asia,"Tiger Global Management, Sequoia Capital China...",100.00000,$2B,E-commerce & direct-to-consumer,2.000000e+09
Stripe,2014-01-23,2010.0,San Francisco,United States,North America,"Khosla Ventures, LowercaseCapital, capitalG",95.00000,$2B,FinTech,2.000000e+09
Klarna,2011-12-12,2005.0,Stockholm,Sweden,Europe,"Institutional Venture Partners, Sequoia Capita...",6.70000,$4B,Fintech,4.000000e+09


In [13]:
def reconvert_extracted_numeric_value(number):
    if np.isnan(number):
        return np.nan
    suffix = ''
    if number >= 1e9:
        number /= 1e9
        suffix = 'B'
    elif number >= 1e6:
        number /= 1e6
        suffix = 'M'
    return f"${number:.1f}{suffix}"

merged_df["Funding ($B)"] = merged_df['Funding_Num'].apply(lambda x: reconvert_extracted_numeric_value(x))

merged_df["Funding ($B)"].head()


Company
Bytedance    $8.0B
SpaceX       $7.0B
SHEIN        $2.0B
Stripe       $2.0B
Klarna       $4.0B
Name: Funding ($B), dtype: object

In [14]:
merged_df.head()

,Date Joined,Year Founded,City,Region,Continent,Select Investors,Valuation ($B),Funding ($B),Industry,Funding_Num
Company,,,,,,,,,,
Bytedance,2017-04-07,2012.0,Beijing,China,Asia,"Sequoia Capital China, SIG Asia Investments, S...",3.22015,$8.0B,Artificial intelligence,8.000000e+09
SpaceX,2012-12-01,2002.0,Hawthorne,United States,North America,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.00000,$7.0B,Other,7.000000e+09
SHEIN,2018-07-03,2008.0,Shenzhen,China,Asia,"Tiger Global Management, Sequoia Capital China...",100.00000,$2.0B,E-commerce & direct-to-consumer,2.000000e+09
Stripe,2014-01-23,2010.0,San Francisco,United States,North America,"Khosla Ventures, LowercaseCapital, capitalG",95.00000,$2.0B,FinTech,2.000000e+09
Klarna,2011-12-12,2005.0,Stockholm,Sweden,Europe,"Institutional Venture Partners, Sequoia Capita...",6.70000,$4.0B,Fintech,4.000000e+09


In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1241 entries, Bytedance to Unstoppable Domains
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date Joined       1241 non-null   object 
 1   Year Founded      1241 non-null   float64
 2   City              1223 non-null   object 
 3   Region            1241 non-null   object 
 4   Continent         1077 non-null   object 
 5   Select Investors  1241 non-null   object 
 6   Valuation ($B)    1241 non-null   float64
 7   Funding ($B)      1241 non-null   object 
 8   Industry          1241 non-null   object 
 9   Funding_Num       1241 non-null   float64
dtypes: float64(3), object(7)
memory usage: 106.6+ KB


In [17]:
merged_df.head()

,Date Joined,Year Founded,City,Region,Continent,Select Investors,Valuation ($B),Funding ($B),Industry,Funding_Num
Company,,,,,,,,,,
Bytedance,2017-04-07,2012.0,Beijing,China,Asia,"Sequoia Capital China, SIG Asia Investments, S...",3.22015,$8.0B,Artificial intelligence,8.000000e+09
SpaceX,2012-12-01,2002.0,Hawthorne,United States,North America,"Founders Fund, Draper Fisher Jurvetson, Rothen...",127.00000,$7.0B,Other,7.000000e+09
SHEIN,2018-07-03,2008.0,Shenzhen,China,Asia,"Tiger Global Management, Sequoia Capital China...",100.00000,$2.0B,E-commerce & direct-to-consumer,2.000000e+09
Stripe,2014-01-23,2010.0,San Francisco,United States,North America,"Khosla Ventures, LowercaseCapital, capitalG",95.00000,$2.0B,FinTech,2.000000e+09
Klarna,2011-12-12,2005.0,Stockholm,Sweden,Europe,"Institutional Venture Partners, Sequoia Capita...",6.70000,$4.0B,Fintech,4.000000e+09
